In [1]:
# importing labraries
import numpy as np
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy.optimize import brentq, curve_fit, least_squares
from scipy.stats import norm
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

> # PART III

In [2]:
from scipy.integrate import quad

## Import excel data and import PART 2 SABR calibration results

In [3]:
# import excel data
discount = pd.read_csv('zero_rates_20201201.csv')
spx_df = pd.read_csv('SPX_options.csv')
spy_df = pd.read_csv('SPY_options.csv')
S0_spx = 3662.45
S0_spy = 366.02
# import part 2 SABR calibration results
sabr_spx = pd.read_csv('sabr_spx.csv')
sabr_spy = pd.read_csv('sabr_spy.csv')

In [4]:
sabr = pd.DataFrame({'SPX T=45': sabr_spx.iloc[1, 1:].values, 
                    'SPY T=45': sabr_spy.iloc[1, 1:].values})
sabr.index = ['𝛼', '𝛽', '𝜌', '𝜈']
sabr

,SPX T=45,SPY T=45
𝛼,1.805269,0.908142
𝛽,0.700000,0.700000
𝜌,-0.390537,-0.488776
𝜈,2.800936,2.728522


## Option Pricing models (T = 45)
1. BlackSholes
2. Bachelier
3. SABR

In [5]:
# define Black Scholes Lognormal pricing functions
def Black_Scholes(S, K, r, sigma, T, opt):
    d1 = (math.log(S / K) + (r + (sigma ** 2) / 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    if opt == 'C':
        return (S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2))
    if opt == 'P':
        return (-S * norm.cdf(-d1) + K * math.exp(-r * T) * norm.cdf(-d2))

# Bachelier Normal pricing functions
def Bachelier(S, K, r, sigma, T, opt):
    d = (K - S) / (sigma * np.sqrt(T)) # why must times S to make things right? 
    if opt == 'C':
        return math.exp(-r * T) *\
        ((S - K) * norm.cdf(-d) + sigma * math.sqrt(T) * norm.pdf(-d))
    if opt == 'P':
        return math.exp(-r * T) *\
        ((K - S) * norm.cdf(d) + sigma * math.sqrt(T) * norm.pdf(d))

def impliedVolatility(S, K, r, price, T, opt, func):
    try:
        impliedVol = brentq(lambda x: price -
                            func(S, K, r, x, T, opt),
                            1e-6, 12000)
    except Exception:
        impliedVol = np.nan

    return impliedVol

def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

def integrand_test(S, K, r, sigma, T, func):
    F = S * np.exp(r*T)    
    if K > F:
        price = func(S, K, r, sigma, T, 'C') / K**2
    else:
        price = func(S, K, r, sigma, T, 'P') / K**2
    return price



Here we try to estimate on discount factors for future use.

Note that the interest rates provided in the 'rate' column are in % unit. So for instance, to discount a cashflow paid 49 days from today, the discount factor is

\begin{equation*}
\begin{split}
D(0,T) = e^{-0.00216480 \times \frac{49}{365}}
\end{split}
\end{equation*}


If the payment date is not provided in the dataframe, you can perform linear interpolation for the corresponding zero rate.

In [6]:
def get_smile_df(df, discount_df, S0, func):
    
    df_cal = df.copy()
    df_cal['T'] =\
            df_cal['exdate']\
            .apply(lambda x: \
                   (pd.Timestamp(str(x))
                   .date() - dt.date(2020, 12, 1))
                   .days)

    df_cal['r'] =\
            df_cal['T']\
            .apply(lambda x: \
                   np.interp(x, discount_df['days'], discount_df['rate'])/100)

    df_cal['F'] =\
            S0 * np.exp(df_cal['r'] * df_cal['T']/365)
    
    # filter out in-the money options and organise dataframes
    # call 
    call = df_cal[df_cal['cp_flag']=='C']
    call['mid_price'] = (call['best_bid'] + call['best_offer']) / 2    
    call = call[call['strike_price']/1000 >= S0]
    
    # put    
    put = df_cal[df_cal['cp_flag']=='P']
    put['mid_price'] = (put['best_bid'] + put['best_offer']) / 2
    put = put[put['strike_price']/1000 <= S0]   
    
    df_smile = pd.concat([put, call])  
    
    df_smile['vol_m'] =\
        df_smile.apply(lambda x:\
                     impliedVolatility(S0,
                                       x['strike_price']/1000,
                                       x['r'],
                                       x['mid_price'],
                                       x['T']/365,
                                       x['cp_flag'],
                                       func),
                          axis=1)
    # df_smile will contain na values, so will smiles
    smiles = []
    for F0 in df_smile['F'].unique():
        smiles += [df_smile[df_smile['F'] == F0]]      
    
    return smiles

In [7]:
spx_smile = get_smile_df(spx_df, discount, S0_spx, Black_Scholes)
spy_smile = get_smile_df(spy_df, discount, S0_spy, Black_Scholes)

In [8]:
spx_smile[1]

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,T,r,F,mid_price,vol_m
1191,20201201,20210115,P,100000,0.0,0.10,E,45,0.002051,3663.376249,0.050,3.020479
1192,20201201,20210115,P,200000,0.0,0.10,E,45,0.002051,3663.376249,0.050,2.408988
1193,20201201,20210115,P,300000,0.0,0.10,E,45,0.002051,3663.376249,0.050,2.062602
1194,20201201,20210115,P,400000,0.0,0.10,E,45,0.002051,3663.376249,0.050,1.821174
1195,20201201,20210115,P,500000,0.0,0.10,E,45,0.002051,3663.376249,0.050,1.636102
...,...,...,...,...,...,...,...,...,...,...,...,...
1186,20201201,20210115,C,5000000,0.0,0.15,E,45,0.002051,3663.376249,0.075,0.276282
1187,20201201,20210115,C,5100000,0.0,0.15,E,45,0.002051,3663.376249,0.075,0.292255
1188,20201201,20210115,C,5200000,0.0,0.15,E,45,0.002051,3663.376249,0.075,0.307836
1189,20201201,20210115,C,5300000,0.0,0.15,E,45,0.002051,3663.376249,0.075,0.323046


## ATM Volatility
Here we implement the solver to find the at-the-money implied volatility

In [9]:
def ATM_Vol_part3(df, S0):
    ATM_slice = df.copy()
    # Calculate the absolute differences between the target value and each row
    ATM_slice['Difference'] = abs(ATM_slice['strike_price']/1000 - S0)

    # Sort the DataFrame by the 'Difference' column
    ATM_slice = ATM_slice.sort_values(by='Difference').head(5)
#     print(ATM_slice)

    # Get the ONLY ONE closest row    
    K = ATM_slice.iloc[0]['strike_price']/1000
    r = ATM_slice.iloc[0]['r']
    price = ATM_slice.iloc[0]['mid_price']
    T = ATM_slice.iloc[0]['T']/365
    opt = ATM_slice.iloc[0]['cp_flag']
    sigma_N = impliedVolatility(S0, K, r, price, T, opt, Bachelier)    
    sigma_LN = impliedVolatility(S0, K, r, price, T, opt, Black_Scholes)
    return [sigma_LN, sigma_N]

In [10]:
sigma_atm = pd.DataFrame({'SPX T=45': ATM_Vol_part3(spx_smile[1], S0_spx),
                         'SPY T=45': ATM_Vol_part3(spy_smile[1], S0_spy)})
sigma_atm.index = ['sigma_BS', 'sigma_Bach']
sigma_atm

,SPX T=45,SPY T=45
sigma_BS,0.188860,0.197767
sigma_Bach,688.146018,72.049480


In [11]:
sigma_atm.to_csv('sigma_atm.csv')

## Integrated Variance by different models
1. BlackSholes
2. Bachelier
3. SABR

## Testing on intergrand() function by way of integrated variance

σ is implied volitality calculated by Black-Scholes model
for testing our integrated variance function

In [12]:
sigma = sigma_atm.loc['sigma_BS', 'SPX T=45']
sigma

0.18886028486850842

In [13]:
S = S0_spx
r = spx_smile[1]['r'].unique()[0]
T = spx_smile[1]['T'].unique()[0]/365
sigma = sigma_atm.loc['sigma_BS', 'SPX T=45']
inf = 500000
I = quad(lambda x: integrand_test(S, x, r, sigma, T, Black_Scholes), 0.0, inf)
E_var = 2*np.exp(r*T)*(I[0])
print('The expected integrated variance is: %.9f' % E_var)
if np.sqrt(E_var/T)-sigma < 1e-6:
    print('Model-free integrated variance model output tally with inputs')

The expected integrated variance is: 0.004397448
Model-free integrated variance model output tally with inputs


In [14]:
def h(K):
    if abs(K) < 1e-12:
        K = 1e-12
    return K**(1/3) + 1.5 * np.log(K) + 10
def h_dif1(K):
    if abs(K) < 1e-12:
        K = 1e-12
    # For testing only
    return 1/3 * K**(-2/3) + 3 / 2 *K**(-1)
def h_dif2(K):
    if abs(K) < 1e-12:
        K = 1e-12
    return -2/9 * K**(-5/3) - 3/2 * K**(-2)

In [15]:
I1 = quad(lambda x: h_dif1(x), 0.0, 50000)
if I1[0] - (h(inf)-h(0)) < 1e-6:
    print('First differencing function checked')
I2 = quad(lambda x: h_dif2(x), 100, 50000)
if I2[0] - (h_dif1(inf)-h_dif1(100)) < 1e-3:
    print('Second differencing function checked')   

First differencing function checked
Second differencing function checked


## Black Scholes Model Derivative Pricing

In [16]:
def integrand(S, K, r, sigma, T, func):
    F = S * np.exp(r*T) 
    if K > F:
        price = func(S, K, r, sigma, T, 'C')
    else:
        price = func(S, K, r, sigma, T, 'P')
    return price * h_dif2(K)

In [38]:
def Static_Replication(df, S, sigma, func):
    r = df['r'].unique()[0]
    T = df['T'].unique()[0]/365

    F = S * np.exp(r*T) 
    inf = 50000    
    I = quad(lambda x: integrand(S, x, r, sigma, T, func), 1e-12, 50000)
    
    return I[0] + h(F)*np.exp(-r*T)
    

In [39]:
sigma_atm

,SPX T=45,SPY T=45
sigma_BS,0.188860,0.197767
sigma_Bach,688.146018,72.049480


In [40]:
Static_spx_BS = Static_Replication(spx_smile[1], 
                                   S0_spx, 
                                   sigma_atm.loc['sigma_BS', 'SPX T=45'], 
                                   Black_Scholes)
Static_spx_BS

37.70444937954976

In [41]:
Static_spy_BS = Static_Replication(spy_smile[1], 
                                   S0_spy, 
                                   sigma_atm.loc['sigma_BS', 'SPY T=45'], 
                                   Black_Scholes)
Static_spy_BS

25.994212271358585

## Bachelier Model Derivative Pricing

In [42]:
sigma = sigma_atm.loc['sigma_Bach', 'SPX T=45']
sigma

688.1460176672617

In [43]:
S = S0_spx
r = spx_smile[1]['r'].unique()[0]
T = spx_smile[1]['T'].unique()[0]/365
sigma = sigma_atm.loc['sigma_Bach', 'SPX T=45']
inf = 500000
Bachelier(S0_spy, 335, r, sigma, T, 'P')

81.6567386969997

In [44]:
Static_spx_Bachelier = Static_Replication(spx_smile[1], 
                                          S0_spx, 
                                          sigma_atm.loc['sigma_Bach', 'SPX T=45'], 
                                          Bachelier)
Static_spx_Bachelier

37.704500600027224

In [45]:
Static_spy_Bachelier = Static_Replication(spy_smile[1], 
                                          S0_spy, 
                                          sigma_atm.loc['sigma_Bach', 'SPY T=45'], 
                                          Bachelier)
Static_spy_Bachelier

25.994234152437546

## SABR Model Derivative Pricing

In [46]:
def integrand_sabr(S, K, r, T, alpha, beta, rho, nu):
    F = S * np.exp(r*T)
    sigma_sabr = SABR(F, K, T, alpha, beta, rho, nu)
    if K > F:
        price = Black_Scholes(S, K, r, sigma_sabr, T, 'C')
    else:
        price = Black_Scholes(S, K, r, sigma_sabr, T, 'P')
    return price * h_dif2(K)


In [47]:
def Static_Replication_SABR(df, S, sabr_results):
    r = df['r'].unique()[0]
    T = df['T'].unique()[0]/365
    F = S * np.exp(r*T) 
    alpha = sabr_results[0]
    beta = sabr_results[1]
    rho = sabr_results[2]
    nu = sabr_results[3]
    
    inf = 50000    
    I = quad(lambda x: integrand_sabr(S, x, r, T, \
                                      alpha, beta, rho, nu), 1, 50000)
    
    return I[0] + h(F)*np.exp(-r*T)

In [48]:
sabr

,SPX T=45,SPY T=45
𝛼,1.805269,0.908142
𝛽,0.700000,0.700000
𝜌,-0.390537,-0.488776
𝜈,2.800936,2.728522


In [49]:
sabr.iloc[:, 0]

𝛼    1.805269
𝛽    0.700000
𝜌   -0.390537
𝜈    2.800936
Name: SPX T=45, dtype: float64

In [50]:
Static_spx_SABR =\
    Static_Replication_SABR(spx_smile[1], S0_spx, sabr.iloc[:, 0])
Static_spx_SABR

37.70044771249642

In [51]:
Static_spy_SABR =\
    Static_Replication_SABR(spy_smile[1], S0_spy, sabr.iloc[:, 1])
Static_spy_SABR

25.992677830470377

## Validating the results (Use SPX option data for T = 45 as examples)

In [56]:
h(S0_spx)

37.723134748877065

Validating the above result on contract h(K) = K^(1/3) + 1.5 * lg(K) + 10:
1. All the three models will give positive call prices C(K) and put prices P(K)
2. h"(K) is always negative for positive underlying asset prices.
3. Price by Static Replication will then be smaller than h(S0)

In [57]:
Static_Replication_all = pd.DataFrame({'SPX T=45' : [Static_spx_BS, 
                                                     Static_spx_Bachelier, 
                                                     Static_spx_SABR],
                                      'SPY T=45' : [Static_spy_BS, 
                                                    Static_spy_Bachelier, 
                                                    Static_spy_SABR]}).T
Static_Replication_all.columns = ['Black-Scholes', 'Bachelier', 'SABR']
Static_Replication_all.to_csv('Part3_pricing.csv')

# Part III (Static Replication) Results

In [58]:
Static_Replication_all

,Black-Scholes,Bachelier,SABR
SPX T=45,37.704449,37.704501,37.700448
SPY T=45,25.994212,25.994234,25.992678
